# Why you should never use Z-Scores

Hi! Here's how using Z-Scores changes the rankings compared to using Relative Marking. I'll first show how the Z-Score itself gives out contradictory data, and I'll also show how relative marking changes things. 

I'm using the final Excel sheet that was released by the ADR Board, converted into CSV format for easy parsing. The same results can be replicated in Excel as well without much difficulty. If you are familiar with Python, you can clone this repo to run this code yourself. 

In [48]:
import pandas as pd
import numpy as np

#MEAN of the each judge, given in the final sheet
ANNMEAN = 45.40740741
KATMEAN = 74.81578947		
JONMEAN = 58.75342466
IRAMEAN = 58.51785714

#STDEV of the each judge, given in the final sheet
ANNSTD = 11.21262271		
KATSTD = 15.24252235
JONSTD = 15.83370847
IRASTD = 22.43163508

In [140]:
#Read Files and clean it up
df = pd.read_csv("nego_tabs1.csv")
df['rank'] = df['rank'].astype('int32')
df['code'] = df['code'].astype('int32')
del df['Unnamed: 2'], df['Unnamed: 4'], df['final']
columns = df.columns

#Recaulcuate the total score based on the Z-Scores (ADR's method)
df['total'] = df['annapurnaZ'] + df['kathleenZ'] + df['johnZ'] + df['iramZ']
df = df.set_index('code')
non_convert = df.copy()

#Display the top 20 (Keep a close eye on the highlighted spot)
df.head(20).style.apply(lambda x: ['background: lightgreen' if x.name in [5] else '' for i in x], axis=1)

,rank,member1,member2,annapurna,annapurnaZ,kathleen,kathleenZ,john,johnZ,iram,iramZ,total
code,,,,,,,,,,,,
76,1,Aditya Wakhlu,Kaustubh Chaturvedi,86.000000,3.620258,98,1.521022,76.000000,1.089232,91.000000,1.448051,7.678563
29,2,Kanav Khanna,Lakshmi Nambiar,63.000000,1.568999,100,1.652234,75.000000,1.026075,94.000000,1.581790,5.829098
6,3,Ananya Patwardhan,Anshul Butani,nan,1.258598,94,1.258598,74.000000,0.962919,93.000000,1.537210,5.017326
65,4,Shreyas Sinha,Prakhar Saunakiya,48.000000,0.231221,93,1.192992,91.000000,2.036578,92.000000,1.492631,4.953421
25,5,Aarohi Chaudhuri,Manas Agrawal,63.000000,1.568999,75,0.012085,nan,1.581790,94.000000,1.581790,4.744665
38,6,Arth Doshi,Arushi Tiwari,nan,0.996174,90,0.996174,74.000000,0.962919,93.000000,1.537210,4.492478
42,7,Anusha Sarkar,Ananya Chaturvedi,nan,0.930568,89,0.930568,76.000000,1.089232,92.000000,1.492631,4.442999
39,8,Abhiram Nitin,Hayden D'Souza,44.000000,-0.125520,86,0.733751,91.000000,2.036578,92.000000,1.492631,4.137439
3,9,Lahar Jain,Sanyukta Fauzdar,58.000000,1.123073,93,1.192992,85.000000,1.657639,53.000000,-0.245986,3.727719


These are the top 20 scores as calculated by the ADR Board. Do note the current rankings, along with the highlighted team. 

# What the heck are Z-Scores?

Z-Scores are calculated using a simple formula, **z = (x-μ)/σ**, where x is the *raw score*, μ is the *population mean*, and σ is the *population standard deviation*. Given the Z-Score, we can easily find out what the original score was supposed to be. When the ADR Board assigned the same Z-Score in cases where a judge was absent, they've implicitly given the person a score that we can easily calculate. Given that the Z-Score only calculates deviation, the results are highly arbitrary.

When you sort the sum of absolute scores, as recreated from the Z-Scores, you get a completely different internal ranking. Plus, one team is replaced with another!


In [156]:
#Based on the Given Z-Scores, recalculate the original score using the formula z = (x-μ)/σ, where x is the raw score, μ is the population mean, and σ is the population standard deviation. Round the result.  

df['annapurna'] = ((df['annapurnaZ'] * ANNSTD) + ANNMEAN).round()
df['kathleen'] = ((df['kathleenZ'] * KATSTD) + KATMEAN).round()
df['john'] = ((df['johnZ'] * JONSTD) + JONMEAN).round()
df['iram'] = ((df['iramZ'] * IRASTD) + IRAMEAN).round()

#Add up the original scores without normalization and re-sort the top 20. Why is there a discrepancy when adding up Z-Scores vs. adding the absolute scores! That's because you're not allowed to add Z-Scores. 
cols = df.columns
df['norm_total'] = df['annapurna'] + df['kathleen'] + df['john'] + df['iram']
df.sort_values('norm_total', ascending=False)[:20].style.apply(lambda x: ['background: yellow' if x.name in [54] else '' for i in x], axis=1)


,rank,member1,member2,annapurna,annapurnaZ,kathleen,kathleenZ,john,johnZ,iram,iramZ,total,norm_total
code,,,,,,,,,,,,,
76,1,Aditya Wakhlu,Kaustubh Chaturvedi,86.000000,3.620258,98.000000,1.521022,76.000000,1.089232,91.000000,1.448051,7.678563,351.000000
29,2,Kanav Khanna,Lakshmi Nambiar,63.000000,1.568999,100.000000,1.652234,75.000000,1.026075,94.000000,1.581790,5.829098,332.000000
65,4,Shreyas Sinha,Prakhar Saunakiya,48.000000,0.231221,93.000000,1.192992,91.000000,2.036578,92.000000,1.492631,4.953421,324.000000
6,3,Ananya Patwardhan,Anshul Butani,60.000000,1.258598,94.000000,1.258598,74.000000,0.962919,93.000000,1.537210,5.017326,321.000000
25,5,Aarohi Chaudhuri,Manas Agrawal,63.000000,1.568999,75.000000,0.012085,84.000000,1.581790,94.000000,1.581790,4.744665,316.000000
38,6,Arth Doshi,Arushi Tiwari,57.000000,0.996174,90.000000,0.996174,74.000000,0.962919,93.000000,1.537210,4.492478,314.000000
42,7,Anusha Sarkar,Ananya Chaturvedi,56.000000,0.930568,89.000000,0.930568,76.000000,1.089232,92.000000,1.492631,4.442999,313.000000
39,8,Abhiram Nitin,Hayden D'Souza,44.000000,-0.125520,86.000000,0.733751,91.000000,2.036578,92.000000,1.492631,4.137439,313.000000
24,10,Aditya Sarma,Debditya Saha,40.000000,-0.482261,93.000000,1.192992,82.000000,1.468170,92.000000,1.492631,3.671532,307.000000


# Here's what they should have actually done

Instead of taking Z-Scores, the ADR Board should have calculated relative scores, borrowing the other judge's relative score when one judge was unavailable. This results in yet another ranking, which is actually the **correct one**.  

In [157]:
df2 = non_convert.copy(deep=True) #Create copy of the OG data

#Relative the scores. This is a simple process. I simply divide the obtained score with the maximum given by that judge. In case a judge is absent, I take the relative score of the other judge. Repeat process for all 4 judges.
df2.loc[~df2['annapurna'].isna(), 'annapurna'] = df2[~df2['annapurna'].isna()]['annapurna']/df2['annapurna'].max()
df2.loc[df2['annapurna'].isna(), 'annapurna'] = df2[df2['annapurna'].isna()]['kathleen']/df2['kathleen'].max()

df2.loc[~df2['kathleen'].isna(), 'kathleen'] = df2[~df2['kathleen'].isna()]['kathleen']/df2['kathleen'].max()
df2.loc[df2['kathleen'].isna(), 'kathleen'] = df2[df2['kathleen'].isna()]['annapurna']/df2['annapurna'].max()

df2.loc[~df2['john'].isna(), 'john'] = df2[~df2['john'].isna()]['john']/df2['john'].max()
df2.loc[df2['john'].isna(), 'john'] = df2[df2['john'].isna()]['iram']/df2['iram'].max()
  
df2.loc[~df2['iram'].isna(), 'iram'] = df2[~df2['iram'].isna()]['iram']/df2['iram'].max()
df2.loc[df2['iram'].isna(), 'iram'] = df2[df2['iram'].isna()]['john']/df2['john'].max()

#Add the relative scores together and sort the top 20
df2['relative_total'] = df2['annapurna'] + df2['kathleen'] + df2['john'] + df2['iram']
df2 = df2.sort_values('relative_total', ascending=False)
df2[:20].style.apply(lambda x: ['background: yellow' if x.name in [54] else '' for i in x], axis=1)

,rank,member1,member2,annapurna,annapurnaZ,kathleen,kathleenZ,john,johnZ,iram,iramZ,total,relative_total
code,,,,,,,,,,,,,
76,1,Aditya Wakhlu,Kaustubh Chaturvedi,1.000000,3.620258,0.980000,1.521022,0.835165,1.089232,0.957895,1.448051,7.678563,3.773060
6,3,Ananya Patwardhan,Anshul Butani,0.940000,1.258598,0.940000,1.258598,0.813187,0.962919,0.978947,1.537210,5.017326,3.672134
38,6,Arth Doshi,Arushi Tiwari,0.900000,0.996174,0.900000,0.996174,0.813187,0.962919,0.978947,1.537210,4.492478,3.592134
42,7,Anusha Sarkar,Ananya Chaturvedi,0.890000,0.930568,0.890000,0.930568,0.835165,1.089232,0.968421,1.492631,4.442999,3.583586
29,2,Kanav Khanna,Lakshmi Nambiar,0.732558,1.568999,1.000000,1.652234,0.824176,1.026075,0.989474,1.581790,5.829098,3.546208
25,5,Aarohi Chaudhuri,Manas Agrawal,0.732558,1.568999,0.750000,0.012085,0.989474,1.581790,0.989474,1.581790,4.744665,3.461506
65,4,Shreyas Sinha,Prakhar Saunakiya,0.558140,0.231221,0.930000,1.192992,1.000000,2.036578,0.968421,1.492631,4.953421,3.456561
52,12,Pallavi Khatri,Vrishank Singhania,0.730000,-0.119127,0.730000,-0.119127,0.989011,1.973421,1.000000,1.626370,3.361538,3.449011
39,8,Abhiram Nitin,Hayden D'Souza,0.511628,-0.125520,0.860000,0.733751,1.000000,2.036578,0.968421,1.492631,4.137439,3.340049


Here's the final top-20 for the preliminary rounds. I'm certain that the top 20 ranks in the final pool will also change if the right method is used. But I leave that as an exercsie for the reader. 

In [158]:
df2['rank'] = range(1, len(df2)+1)
df2.head(20)[cols[:3]].style.apply(lambda x: ['background: yellow' if x.name in [54] else '' for i in x], axis=1)

,rank,member1,member2
code,,,
76,1,Aditya Wakhlu,Kaustubh Chaturvedi
6,2,Ananya Patwardhan,Anshul Butani
38,3,Arth Doshi,Arushi Tiwari
42,4,Anusha Sarkar,Ananya Chaturvedi
29,5,Kanav Khanna,Lakshmi Nambiar
25,6,Aarohi Chaudhuri,Manas Agrawal
65,7,Shreyas Sinha,Prakhar Saunakiya
52,8,Pallavi Khatri,Vrishank Singhania
39,9,Abhiram Nitin,Hayden D'Souza


And this is the final rank-list. It seems that one team has been denied the opportunity to participate in the finals, at the expense of another team. While I have no truck with the Board, I assume that this team might. 

Thanks for reading!